<a href="https://colab.research.google.com/github/Dagger77/amzn_recsys/blob/main/eda_notebooks/Amazon_Reviews_2018_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scikit-surprise -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
pip install lightfm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##IMPORT

In [4]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import itertools
from tqdm import tqdm
import time
import numpy as np
import operator
from itertools import combinations
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold, GridSearchCV
from surprise.prediction_algorithms import *
from sklearn.preprocessing import LabelEncoder

In [6]:
random_seed = 42

##LOAD

In [49]:
df = pd.read_parquet('/content/drive/MyDrive/Velotix HA/amazon_processed.parquet')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 510312 entries, 0 to 551158
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   userName          510312 non-null  object 
 1   itemName          510312 non-null  object 
 2   description       510312 non-null  object 
 3   brand             510312 non-null  object 
 4   category          510312 non-null  object 
 5   price             510312 non-null  float64
 6   rating            510312 non-null  float64
 7   summary           510312 non-null  object 
 8   brand_encoded     510312 non-null  int64  
 9   category_encoded  510312 non-null  int64  
 10  user_id           510312 non-null  int64  
 11  item_id           510312 non-null  int64  
 12  desc_emb          510312 non-null  object 
 13  sentiment         510312 non-null  object 
dtypes: float64(2), int64(4), object(8)
memory usage: 58.4+ MB


In [51]:
# df['user_id'] = df['user_id'].astype(str)
# df['item_id'] = df['item_id'].astype(str)
df['brand_encoded'] = df['brand_encoded'].astype(str)
df['category_encoded'] = df['category_encoded'].astype(str)

##SMALL PREPROC

In [52]:
#Let's filter out brands that are not so popular
brand_counts = df['brand'].value_counts()
popular_brands = brand_counts[brand_counts > 500].index
df['brand_filtered'] = df['brand'].apply(lambda x: x if x in popular_brands else 'other')

le = LabelEncoder()
df['brand_encoded_filtered'] = le.fit_transform(df['brand_filtered'])
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))

print(label_mapping)
print(df[['brand', 'brand_filtered', 'brand_encoded_filtered']].head(10))

{'3M': 0, 'API': 1, 'AUKEY': 2, 'AmazonBasics': 3, 'Anker': 4, "Anthony's": 5, 'Aqueon': 6, 'Arm & Hammer': 7, 'Avery': 8, 'BIC': 9, 'BLACK+DECKER': 10, 'Bayer Animal Health': 11, 'Best Bully Sticks': 12, 'Best Pet Supplies, Inc.': 13, 'Birch Benders': 14, 'Blue Buffalo': 15, 'Blueberry Pet': 16, "Bob's Red Mill": 17, 'Briggs & Stratton': 18, 'Brother': 19, 'Canon': 20, 'Catit': 21, 'Chuckit!': 22, 'Classic Accessories': 23, 'Clover': 24, "D'Addario": 25, 'DYMO': 26, 'Darice': 27, 'Downtown Pet Supply': 28, 'Dritz': 29, 'Epson': 30, 'Ethical Pet': 31, 'Expo': 32, 'Fender': 33, 'Fiskars': 34, 'Fluval': 35, 'Frontier': 36, 'Furminator': 37, 'Go Pet Club': 38, 'GoTags': 39, 'Green Mountain Coffee Roasters': 40, 'Greenies': 41, 'Greenworks': 42, 'HP': 43, 'Hartz': 44, "Hill's Science Diet": 45, 'IRIS USA, Inc.': 46, 'Iams': 47, 'Intex': 48, 'JETech': 49, 'JW Pet': 50, 'K&H Pet Products': 51, 'KIND': 52, 'KONG': 53, 'Kaytee': 54, 'Kirkland Signature': 55, 'MarineLand': 56, 'McCormick': 57, 

In [53]:
#Let's save the mappings
user_to_id_map = dict(zip(df['userName'], df['user_id']))
user_to_name_map = {user_id: user_name for user_name, user_id in user_to_id_map.items()}

item_to_id_map = dict(zip(df['itemName'], df['item_id']))
item_to_name_map = {item_id: item_name for item_name, item_id in item_to_id_map.items()}

##TRAIN/TEST SPLIT

In [12]:
#Let's filter out users and items having small number of interactions
min_user_ratings = 10
min_item_ratings = 10
filtered_data = df.groupby('user_id').filter(lambda x: len(x) >= min_user_ratings)
filtered_data = filtered_data.groupby('item_id').filter(lambda x: len(x) >= min_item_ratings)

In [ ]:
# # Downsample the highly active users
# max_interactions = 500

# # Filter out highly active users and downsample them separately
# amazon_customer_data = df[df['userName'] == 'Amazon Customer'].sample(n=max_interactions, random_state=42)
# kindle_customer_data = df[df['userName'] == 'Kindle Customer'].sample(n=max_interactions, random_state=42)

# # Filter out the data for other users
# other_users_data = df[(df['userName'] != 'Amazon Customer') & (df['userName'] != 'Kindle Customer')]

# # Combine the downsampled data with the other users' data
# filtered_data = pd.concat([other_users_data, amazon_customer_data, kindle_customer_data])

In [13]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67503 entries, 1 to 551150
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   userName                67503 non-null  object 
 1   itemName                67503 non-null  object 
 2   description             67503 non-null  object 
 3   brand                   67503 non-null  object 
 4   category                67503 non-null  object 
 5   price                   67503 non-null  float64
 6   rating                  67503 non-null  float64
 7   summary                 67503 non-null  object 
 8   brand_encoded           67503 non-null  object 
 9   category_encoded        67503 non-null  object 
 10  user_id                 67503 non-null  int64  
 11  item_id                 67503 non-null  int64  
 12  desc_emb                67503 non-null  object 
 13  sentiment               67503 non-null  object 
 14  brand_filtered          67503 non-null  ob

In [ ]:
# We need Users to have at least 2 ratings to split train/test
eligible_users = filtered_data['user_id'].value_counts()[filtered_data['user_id'].value_counts() > 1].index
eligible_data = filtered_data[filtered_data['user_id'].isin(eligible_users)]

train_df, test_df = train_test_split(
    eligible_data, test_size=0.2, random_state=random_seed, stratify=eligible_data['user_id']
)

# # Optionally handle users with exactly 1 rating separately
# remaining_data = filtered_data[~filtered_data['user_id'].isin(eligible_users)]
# # adding all to the train set
# train_df = pd.concat([train_df, remaining_data])

In [ ]:
print(len(eligible_data))
print(len(train_df))
print(len(test_df))

66895
53516
13379


In [ ]:
# Ensure that each user in the test set has at least 5 interactions to measure Precision@5
#test_df = test_df.groupby('user_id').filter(lambda x: len(x) >= 5)

In [ ]:
# Ensure all users in test are also in train
train_df = train_df[train_df['user_id'].isin(test_df['user_id'])]

# Ensure all items in test are also in train
train_df = train_df[train_df['item_id'].isin(test_df['item_id'])]

In [ ]:
print(len(train_df))
print(len(test_df))

50316
13379


##SURPRISE ALGOS

In [ ]:
columns = ['user_id', 'item_id', 'rating']
train_df = train_df[columns]
test_df = test_df[columns]

In [ ]:
test_df.head()

,user_id,item_id,rating
334066,26821,6974,5.0
23994,9739,8171,1.0
160086,49507,6232,4.0
12011,5190,1810,4.0
262594,22980,55716,5.0


In [ ]:
print(test_df.user_id.nunique(), test_df.item_id.nunique())

5636 3098


In [ ]:
print(train_df.user_id.nunique(), train_df.item_id.nunique())

5636 3098


###Metric functions

* Precision@K measures how many of the top K items recommended to a user are relevant (i.e., items that the user actually interacted with or rated positively)

* Recall@K measures how many of the total relevant items for a user were actually recommended in the top K recommendations

In [ ]:
def compute_precision(test_set, recommendations):
    #hits is the number of items that were recommended and chosen
    hits = 0
    #recs is the total number of recommended items
    recs = 0

    for u in test_set.user_id.unique():
        userData = test_set[test_set.user_id == u]
        userRecs = recommendations.get(str(u))

        hits += len(set(userData.item_id).intersection(set(userRecs)))
        recs += len(userRecs)

    return hits / recs

In [ ]:
def compute_precision_at_k(test_set, recommendations, k=5):
    hits = 0
    recs = 0

    for u in test_set.user_id.unique():
        userData = test_set[test_set.user_id == u]
        userRecs = recommendations.get(str(u), [])

        top_k_recs = userRecs[:k]

        # hits = intersection between relevant items and top K recommended items
        hits += len(set(userData.item_id).intersection(set(top_k_recs)))

        # Increment recs by K or fewer if fewer recommendations are available
        recs += min(len(top_k_recs), k)

    # precision at K = hits divided by total recommendations at K
    return hits / recs if recs > 0 else 0

In [ ]:
def compute_recall(test_set, recommendations):
    hits = 0
    total_relevant = 0

    for u in test_set.user_id.unique():
        userData = test_set[test_set.user_id == u]
        userRecs = recommendations.get(str(u))

        hits += len(set(userData.item_id).intersection(set(userRecs)))
        total_relevant += len(userData)

    return hits / total_relevant

In [ ]:
def compute_recall_at_k(test_set, recommendations, k=5):
    hits = 0
    total_relevant = 0

    for u in test_set.user_id.unique():
        userData = test_set[test_set.user_id == u]
        userRecs = recommendations.get(str(u), [])

        top_k_recs = userRecs[:k]

        # hits = intersection between relevant items and top K recommended items
        hits += len(set(userData.item_id).intersection(set(top_k_recs)))

        #items user interacted with in the test set
        total_relevant += len(userData)

    # Return recall@k: hits divided by total relevant items
    return hits / total_relevant if total_relevant > 0 else 0

###Baseline

In [ ]:
# Most popular item recommender
class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):
        self.train_set = train_set

        self.item_ratings_sorted = (self.train_set
                                    .groupby('item_id')
                                    .user_id.size()
                                    .sort_values(ascending=False)
                                    .reset_index()
                                    .item_id
                                    .to_list()
                                    )

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}
        already_ranked_items_by_user = self.train_set.groupby('user_id')['user_id'].apply(list)

        #For each user in the test set compute recommendations
        for user_id in test_set.user_id.unique():
            ranked = set(already_ranked_items_by_user[user_id])
            result[str(user_id)] = [item for item in self.item_ratings_sorted if item not in ranked][:top_n]

        return result

In [ ]:
# Class for Surprise library algos
class SurpriseRecMethod():

    #method will be the specific Surprise algorithm that we will call
    def __init__(self, method):
        self.method = method

    def fit(self, train_set):
        self.train_set = train_set


    def get_top_n_recommendations(self, test_set, top_n):
        self.test_set = test_set

        #Surprise requires a slightly different input data format, so we use two different CSVs
        test_path_tmp = "test_file.csv"
        train_path_tmp = "train_file.csv"

        self.train_set.to_csv(train_path_tmp, index=False, header=False)
        self.test_set.to_csv(test_path_tmp, index=False, header=False)

        fold_files = [(train_path_tmp, test_path_tmp)]
        reader = Reader(rating_scale=(1, 10), line_format='user item rating', sep=',')
        data = Dataset.load_from_folds(fold_files, reader=reader)

        for trainset, testset in PredefinedKFold().split(data):
            self.method.fit(trainset)

        already_ranked_items_by_users = self.train_set.groupby('user_id')['item_id'].apply(list)

        recommendations = {}
        pbar = tqdm(total=len(self.test_set.user_id.unique()))
        for user_id in self.test_set.user_id.unique():
            pbar.update(1)

            if user_id not in self.train_set.user_id.unique():
                recommendations[str(user_id)] = []
                continue

            items_expected_ranking = {}
            for item_id in self.train_set.item_id.unique():
                if item_id in already_ranked_items_by_users[user_id]:
                    continue

                #We call here the specific Surprise method that we use for this model
                #The method predicts a score for a given item
                predicted = self.method.predict(str(user_id), str(item_id), clip=False)
                items_expected_ranking[item_id] = predicted.est

            #Now we just sort by decreasing scores and take the top N
            sorted_predictions = sorted(items_expected_ranking.items(), key=operator.itemgetter(1), reverse=True)
            user_recommendations = [x[0] for x in sorted_predictions[:top_n]]
            recommendations[str(user_id)] = user_recommendations

        pbar.close()
        return recommendations

In [ ]:
popular = MostPopular()
popular.learn_model(train_df)
popular_recs = popular.get_top_n_recommendations(test_df,top_n=5)

In [ ]:
p0 = compute_precision_at_k(test_df, popular_recs, 5)
r0 = compute_recall_at_k(test_df, popular_recs, 5)
print(f'Precision@5: {p0:.4f}', f'Recall@5: {r0:.4f}')

Precision@5: 0.0067 Recall@5: 0.0141


###Surpise methods

In [ ]:
modelSlopeOne = SurpriseRecMethod(SlopeOne())
modelSlopeOne.fit(train_df)
recSlopeOne = modelSlopeOne.get_top_n_recommendations(test_df, 5)

100%|██████████| 5636/5636 [04:48<00:00, 19.53it/s]


Precision@5: 0.0006 Recall@5: 0.0013


In [ ]:
p1 = compute_precision_at_k(test_df,recSlopeOne, 5)
r1 = compute_recall_at_k(test_df,recSlopeOne, 5)
print(f'Precision@5: {p1:.4f}', f'Recall@5: {r1:.4f}')

Precision@5: 0.0006 Recall@5: 0.0013


In [ ]:
modelKNNUser = SurpriseRecMethod(KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True}))
modelKNNUser.fit(train_df)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_df, 5)

Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|██████████| 5636/5636 [10:18<00:00,  9.11it/s]


In [ ]:
p2 = compute_precision_at_k(test_df,recKNNUser, 5)
r2 = compute_recall_at_k(test_df,recKNNUser, 5)
print(f'Precision@5: {p2:.4f}', f'Recall@5: {r2:.4f}')

Precision@5: 0.0006 Recall@5: 0.0012


In [ ]:
modelKNNItem = SurpriseRecMethod(KNNWithMeans(sim_options={'name': 'cosine', 'user_based': False}))
modelKNNItem.fit(train_df)
recKNNItem = modelKNNItem.get_top_n_recommendations(test_df, 5)

Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|██████████| 5636/5636 [06:27<00:00, 14.56it/s]


In [ ]:
p3 = compute_precision_at_k(test_df,recKNNItem, 5)
r3 = compute_recall_at_k(test_df,recKNNItem, 5)
print(f'Precision@5: {p3:.4f}', f'Recall@5: {r3:.4f}')

Precision@5: 0.0009 Recall@5: 0.0019


In [ ]:
modelNMF = SurpriseRecMethod(NMF(n_factors=15, n_epochs=50, random_state=random_seed, biased=False))
modelNMF.fit(train_df)
recNMF = modelNMF.get_top_n_recommendations(test_df, 5)

100%|██████████| 5636/5636 [03:51<00:00, 24.31it/s]


In [ ]:
p4 = compute_precision_at_k(test_df,recNMF, 5)
r4 = compute_recall_at_k(test_df,recNMF, 5)
print(f'Precision@5: {p4:.4f}', f'Recall@5: {r4:.4f}')

Precision@5: 0.0007 Recall@5: 0.0016


In [ ]:
modelNMF2 = SurpriseRecMethod(NMF(n_factors=100, n_epochs=50, reg_pu=0.2, reg_qi=0.2, random_state=random_seed, biased=True))
modelNMF2.fit(train_df)
recNMF2 = modelNMF2.get_top_n_recommendations(test_df, 5)

100%|██████████| 5636/5636 [04:06<00:00, 22.82it/s]


In [ ]:
p5 = compute_precision_at_k(test_df,recNMF2, 5)
r5 = compute_recall_at_k(test_df,recNMF2, 5)
print(f'Precision@5: {p5:.4f}', f'Recall@5: {r5:.4f}')

Precision@5: 0.0005 Recall@5: 0.0011


In [ ]:
results_df = pd.DataFrame.from_dict(
    {'Baseline_Popular': [p0, r0],
     'SlopeOne': [p1, r1],
     'User KNN': [p2, r2],
     'Item KNN': [p3, r3],
     'NMF1': [p4, r4],
     'NMF2': [p5, r5]},
    orient='index',
    columns=['Precision@5', 'Recall@5']
)
results_df

,Precision@5,Recall@5
Baseline_Popular,0.006671,0.014052
SlopeOne,0.000639,0.001345
User KNN,0.000568,0.001196
Item KNN,0.000923,0.001943
NMF1,0.000745,0.001570
NMF2,0.000532,0.001121


Very low scores. Seems like we don't have enough data to extract meaningful connections, another factor is the sparcity.

##LightFM

In [14]:
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import coo_matrix, lil_matrix, csr_matrix
from lightfm.data import Dataset
from sklearn.model_selection import StratifiedShuffleSplit
from lightfm.evaluation import precision_at_k, auc_score

In [54]:
min_user_ratings = 10
min_item_ratings = 10
filtered_data = df.groupby('user_id').filter(lambda x: len(x) >= min_user_ratings)
filtered_data = filtered_data.groupby('item_id').filter(lambda x: len(x) >= min_item_ratings)

In [55]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)

# For stratified split we need Users with at least 2 ratings
eligible_users = filtered_data['user_id'].value_counts()[filtered_data['user_id'].value_counts() > 1].index

eligible_data = filtered_data[filtered_data['user_id'].isin(eligible_users)]

stratify_by = eligible_data['user_id']

for train_index, test_index in sss.split(eligible_data, stratify_by):
    train_df = eligible_data.iloc[train_index]
    test_df = eligible_data.iloc[test_index]

In [56]:
# Ensure all users in test are also in train
train_df = train_df[train_df['user_id'].isin(test_df['user_id'])]

# Ensure all items in test are also in train
train_df = train_df[train_df['item_id'].isin(test_df['item_id'])]

In [57]:
print(test_df.user_id.nunique(), test_df.item_id.nunique())

5636 3098


In [58]:
print(train_df.user_id.nunique(), train_df.item_id.nunique())

5636 3098


In [59]:
train_df.shape

(50316, 16)

In [60]:
test_df.shape

(13379, 16)

In [61]:
eligible_data.shape

(66895, 16)

###ONLY INTERACTIONS

In [62]:
dataset = Dataset()
dataset.fit(users=eligible_data['user_id'].unique(), items=eligible_data['item_id'].unique())

# Building the interaction matrices for training
train_interactions, train_weights = dataset.build_interactions(
    zip(train_df['user_id'].values, train_df['item_id'].values, train_df['rating'].values)
)

# Building the interaction matrix for testing
test_interactions, test_weights = dataset.build_interactions(
    zip(test_df['user_id'].values, test_df['item_id'].values, test_df['rating'].values)
)

In [63]:
model_clear = LightFM(no_components=128,  # the dimensionality of the feature latent embeddings
                			learning_schedule='adagrad', # type of optimiser to use
                			loss='warp', # loss type
                			learning_rate=0.02, # set the initial learning rate
                			item_alpha=0.0, # L2 penalty on item features
                			user_alpha=0.0, # L2 penalty on users features
                			max_sampled=10, # maximum number of negative samples used during WARP fitting
                			random_state=random_seed)

model_clear.fit(train_interactions, # our training data
               epochs = 50,
               num_threads = 4,
               verbose=True)

Epoch: 100%|██████████| 50/50 [00:17<00:00,  2.80it/s]


In [64]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [65]:
# Evaluate the model on the test set
test_precision = precision_at_k(model_clear, test_interactions, k=5).mean()
test_auc = auc_score(model_clear, test_interactions).mean()

print(f"Precision@5: {test_precision:.4f}")
print(f"AUC: {test_auc:.4f}")

# Precision@5: 0.0284
# AUC: 0.7246

Precision@5: 0.0285
AUC: 0.7276


Results are better!

###ITEM FEATURES INCLUDED

####Categorical Features: Brand & Category

In [ ]:
# dataset = Dataset()
# # We need to fit all features: price, embedding features, and categorical features
# item_features_list = ['price'] + \
#                      [f'desc_emb_{i}' for i in range(512)] + \
#                      [f'brand_encoded_{val}' for val in eligible_data['brand_encoded'].unique()] + \
#                      [f'category_encoded_{val}' for val in eligible_data['category_encoded'].unique()]

In [ ]:
# Only categorical features experiment
dataset = Dataset()
item_features_list = [f'brand_encoded_filtered_{val}' for val in eligible_data['brand_encoded_filtered'].unique()] + \
                     [f'category_encoded_{val}' for val in eligible_data['category_encoded'].unique()]

In [ ]:
dataset.fit(
    users=eligible_data['user_id'].unique(),
    items=eligible_data['item_id'].unique(),
    user_features=None,
    item_features=item_features_list
)

In [ ]:
# Getting mappings
#FROM DOCUMENTATION
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [ ]:
item_brand_dict = eligible_data.set_index('item_id')['brand_encoded_filtered'].to_dict()
item_category_dict = eligible_data.set_index('item_id')['category_encoded'].to_dict()

In [ ]:
# Construct the data for build_item_features
item_features_data = []

for idx, item_id in tqdm(enumerate(eligible_data['item_id']), total=len(eligible_data['item_id'])):
    feature_dict = {}

    # Price feature
    # if 'price' in item_feature_map:
    #     feature_dict['price'] = item_price_dict[item_id]

    #Categorical features
    for feature, item_dict in [('brand_encoded_filtered', item_brand_dict), ('category_encoded', item_category_dict)]:
        value = item_dict[item_id]
        feature_name = f'{feature}_{value}'
        if feature_name in item_feature_map:
            feature_dict[feature_name] = 1  # Using binary encoding for categorical features

    item_features_data.append((item_id, feature_dict))

# Use dataset.build_item_features to construct the feature matrix
item_features = dataset.build_item_features(item_features_data, normalize=True)

100%|██████████| 66895/66895 [00:00<00:00, 309523.35it/s]


In [ ]:
item_features.shape

(3246, 3381)

In [ ]:
# Building the interaction matrices for training
train_interactions, train_weights = dataset.build_interactions(
    zip(train_df['user_id'].values, train_df['item_id'].values, train_df['rating'].values)
)

# Building the interaction matrix for testing
test_interactions, test_weights = dataset.build_interactions(
    zip(test_df['user_id'].values, test_df['item_id'].values, test_df['rating'].values)
)

####Hyperparameters tuning

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.metrics import make_scorer
class LightFMWrapper(BaseEstimator):
    def __init__(self, no_components=30, learning_schedule='adagrad', loss='warp', learning_rate=0.05,
                 item_alpha=0.0, user_alpha=0.0, max_sampled=10, random_state=None, epochs=50):
        self.no_components = no_components
        self.learning_schedule = learning_schedule
        self.loss = loss
        self.learning_rate = learning_rate
        self.item_alpha = item_alpha
        self.user_alpha = user_alpha
        self.max_sampled = max_sampled
        self.random_state = random_state
        self.epochs = epochs
        self.model = LightFM(no_components=no_components,
                             learning_schedule=learning_schedule,
                             loss=loss,
                             learning_rate=learning_rate,
                             item_alpha=item_alpha,
                             user_alpha=user_alpha,
                             max_sampled=max_sampled,
                             random_state=random_state)

    def fit(self, interactions, item_features=None, epochs=None):
        if epochs is None:
            epochs = self.epochs
        self.model.fit(interactions, item_features=item_features, epochs=epochs, num_threads=4)
        return self

    def score(self, interactions, item_features=None):
        return precision_at_k(self.model, interactions, item_features=item_features, k=5, num_threads=4).mean()

In [ ]:
# param_grid = {
#     'no_components': [32, 64, 128],
#     'learning_schedule': ['adagrad'],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'item_alpha': [0.0, 0.01, 0.1],
#     'user_alpha': [0.0],
#     'max_sampled': [5, 10, 15],
#     'epochs': [20, 30, 50]
# }

In [ ]:
# TESTED FOR CATEGORICAL FEATURE SET ONLY.
# from sklearn.model_selection import GridSearchCV

# grid_search = GridSearchCV(estimator=LightFMWrapper(random_state=42, loss='warp', epochs=30),
#                            param_grid=param_grid,
#                            verbose=3,
#                            n_jobs=-1,
#                            cv=3
#                            )

# grid_search.fit(train_interactions, item_features=item_features)
# print("Best parameters found: ", grid_search.best_params_)

#Best parameters found:  {'epochs': 30, 'item_alpha': 0.0, 'learning_rate': 0.1, 'learning_schedule': 'adagrad', 'max_sampled': 5, 'no_components': 64, 'user_alpha': 0.0}

In [ ]:
model = LightFM(no_components=64,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=0.1,
                item_alpha=0.0,
                user_alpha=0.0,
                max_sampled=5,
                random_state=random_seed)

model.fit(train_interactions,
          item_features=item_features,
          epochs=30,
          num_threads=4,
          verbose=True)

Epoch: 100%|██████████| 30/30 [00:08<00:00,  3.51it/s]


In [ ]:
start_time = time.time()
test_precision = precision_at_k(model, test_interactions, item_features=item_features, k=5).mean()
precision_time = time.time()
test_auc = auc_score(model, test_interactions, item_features=item_features).mean()
auc_time = time.time()

precision_elapsed_time = (precision_time - start_time) / 60
auc_elapsed_time = (auc_time - precision_time) / 60

print(f"Precision@5: {test_precision:.4f}")
print(f"Time taken for Precision@5: {precision_elapsed_time:.0f} minutes")

print(f"AUC: {test_auc:.4f}")
print(f"Time taken for AUC: {auc_elapsed_time:.0f} minutes")

Precision@5: 0.0237
Time taken for Precision@5: 0 minutes
AUC: 0.7570
Time taken for AUC: 0 minutes


Categorical item features don't seem to be very useful. Let's try description embeddings

####Description embeddings

In [ ]:
# Only embeddings experiment
dataset = Dataset()
item_features_list = [f'desc_emb_{i}' for i in range(512)]

In [ ]:
dataset.fit(
    users=eligible_data['user_id'].unique(),
    items=eligible_data['item_id'].unique(),
    user_features=None,
    item_features=item_features_list
)

In [ ]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
item_embedding_dict = eligible_data.set_index('item_id')['desc_emb'].to_dict()

In [ ]:
# Construct the data for build_item_features
item_features_data = []

for idx, item_id in tqdm(enumerate(eligible_data['item_id']), total=len(eligible_data['item_id'])):
    feature_dict = {}

    # Price feature
    # if 'price' in item_feature_map:
    #     feature_dict['price'] = item_price_dict[item_id]

    # Embeddings
    emb_list = item_embedding_dict[item_id]
    for i, emb_value in enumerate(emb_list):
        feature_name = f'desc_emb_{i}'
        if feature_name in item_feature_map:
            feature_dict[feature_name] = emb_value

    #Categorical features
    # for feature, item_dict in [('brand_encoded_filtered', item_brand_dict), ('category_encoded', item_category_dict)]:
    #     value = item_dict[item_id]
    #     feature_name = f'{feature}_{value}'
    #     if feature_name in item_feature_map:
    #         feature_dict[feature_name] = 1  # Using binary encoding for categorical features

    item_features_data.append((item_id, feature_dict))

# Use dataset.build_item_features to construct the feature matrix
item_features = dataset.build_item_features(item_features_data, normalize=True)

100%|██████████| 66895/66895 [00:38<00:00, 1744.22it/s]


In [ ]:
item_features.shape

(3246, 3758)

In [ ]:
del item_features_data

In [ ]:
model = LightFM(no_components=100,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=0.2,
                item_alpha=0.0,
                user_alpha=0.0,
                max_sampled=5,
                random_state=random_seed)

model.fit(train_interactions,
          item_features=item_features,
          epochs=50,
          num_threads=4,
          verbose=True)

Epoch: 100%|██████████| 50/50 [24:23<00:00, 29.26s/it]


In [ ]:
# Evaluate the model on the test set
start_time = time.time()
test_precision = precision_at_k(model, test_interactions, item_features=item_features, k=5).mean()
precision_time = time.time()
test_auc = auc_score(model, test_interactions, item_features=item_features).mean()
auc_time = time.time()

precision_elapsed_time = (precision_time - start_time) / 60
auc_elapsed_time = (auc_time - precision_time) / 60

# Print the results
print(f"Precision@5: {test_precision:.4f}")
print(f"Time taken for Precision@5: {precision_elapsed_time:.0f} minutes")

print(f"AUC: {test_auc:.4f}")
print(f"Time taken for AUC: {auc_elapsed_time:.0f} minutes")

#Precision@5: 0.0176
#AUC: 0.7441

Precision@5: 0.0180
Time taken for Precision@5: 12 minutes
AUC: 0.7428
Time taken for AUC: 12 minutes


No metric improvements.

##CONCLUSION

* Precision@5 is relatively low for every model. This indicates that the model isn't identifying many relevant items within the top 5 recommendations.
These values are expected in scenarios where the dataset is sparse. Users interact with only a small portion of items.

* The AUC scores are reasonably good. An AUC above 0.7 generally indicates that the model is performing better than random in ranking relevant items higher than irrelevant ones.

* For the sake of performance it's better to choose LightFM model based only on user/items interactions not including features.



##TEST

In [23]:
test_interactions_csr = test_interactions.tocsr()

all_scores = []
for user_id in test_df['user_id']:
    internal_user_id = user_id_map[user_id]
    scores = model_clear.predict(internal_user_id, np.arange(test_interactions.shape[1]))
    all_scores.extend(scores)

min_score = np.min(all_scores)
max_score = np.max(all_scores)

def rescale_score(score, min_score, max_score, new_min=1, new_max=5):
    #rescale the predicted score to the 1-5 range
    return ((score - min_score) / (max_score - min_score)) * (new_max - new_min) + new_min

In [70]:
random_users = np.random.choice(test_df['user_id'].unique(), size=20, replace=False)

for original_user_id in random_users:
    # if original_user_id in user_id_map:
        internal_user_id = user_id_map[original_user_id]
        print(f"User {original_user_id} (internal_id {internal_user_id}),  userName: {user_to_name_map[original_user_id]})")

        # Get the actual items the user interacted with in the test set
        user_interactions = test_df[test_df['user_id'] == original_user_id]

        # Limit to 5 actual interactions
        if len(user_interactions) > 5:
             user_interactions = user_interactions.sample(5, random_state=42)

        actual_items = user_interactions['item_id'].values
        actual_ratings = user_interactions['rating'].values
        print("\n" + "-"*10 + "\n")
        print(f'Interacted Items IDs : {actual_items}')

        # Map actual item IDs to internal indices, skip items not found in the map
        internal_item_ids = []
        filtered_actual_items = []
        filtered_actual_ratings = []

        for item_id, rating in zip(actual_items, actual_ratings):
          print(f'item_id: {item_id}, item_name: {item_to_name_map[item_id]}')

        print("\n" + "-"*10 + "\n")
        if item_id in item_id_map:
                internal_item_ids.append(item_id_map[item_id])
                filtered_actual_items.append(item_id)
                filtered_actual_ratings.append(rating)
        else:
                print(f"Warning: Item ID {item_id} not found in item_id_map. Item is skipped.")

        # if there are valid items after filtering
        if internal_item_ids:

            predicted_scores = model_clear.predict(internal_user_id, internal_item_ids)
            rescaled_scores = [rescale_score(score, min_score, max_score) for score in predicted_scores]

            print(f"Actual vs Predicted Ratings:")
            for item_id, actual_rating, predicted_rating in zip(filtered_actual_items, filtered_actual_ratings, rescaled_scores):
                print(f"Item {item_id} - Actual rating: {actual_rating:.2f}, Predicted rating: {predicted_rating:.2f}")

        print("\n" + "="*50 + "\n")

User 9528 (internal_id 1027),  userName: John B.)

----------

Interacted Items IDs : [8214]
item_id: 8214, item_name: Galaxy S7 Screen Protector Glass (3D Curved Full Screen Coverage), amFilm Bye-Bye-Bubble 3D Curved Samsung Galaxy S7 Tempered Glass Screen Protector [NOT S7 Edge] 2016

----------

Actual vs Predicted Ratings:
Item 8214 - Actual rating: 1.00, Predicted rating: 3.46


User 56786 (internal_id 4167),  userName: Nichols)

----------

Interacted Items IDs : [10424]
item_id: 10424, item_name: Bird Perch, Rope Bungee Bird Toy

----------

Actual vs Predicted Ratings:
Item 10424 - Actual rating: 5.00, Predicted rating: 3.51


User 43768 (internal_id 3390),  userName: certinut)

----------

Interacted Items IDs : [6939]
item_id: 6939, item_name: Kitty City Kit

----------

Actual vs Predicted Ratings:
Item 6939 - Actual rating: 5.00, Predicted rating: 3.44


User 77353 (internal_id 5111),  userName: Sandra Hutchens)

----------

Interacted Items IDs : [13231  5426   414  6736]


##SAVING MODEL

before saving - train the model on the whole dataset

In [86]:
known_interactions = eligible_data.groupby('user_id')['item_id'].apply(set).to_dict()

In [87]:
dataset = Dataset()
dataset.fit(users=eligible_data['user_id'].unique(), items=eligible_data['item_id'].unique())

# Building the interaction matrices for training
interactions, weights = dataset.build_interactions(
    zip(eligible_data['user_id'].values, eligible_data['item_id'].values, eligible_data['rating'].values)
)

In [88]:
model_clear = LightFM(no_components=128,
                			learning_schedule='adagrad',
                			loss='warp',
                			learning_rate=0.02,
                			item_alpha=0.0,
                			user_alpha=0.0,
                			max_sampled=10,
                			random_state=random_seed)

model_clear.fit(interactions,
               epochs = 50,
               num_threads = 4,
               verbose=True)

Epoch: 100%|██████████| 50/50 [00:23<00:00,  2.09it/s]


In [93]:
#We will recomend to interacted items with rating greater than 4 to the unknown user
filtered_df = df[(df['rating'] >= 4)]
top_20_items = filtered_df['item_id'].value_counts().head(20).index.tolist()

In [20]:
import pickle

In [89]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [90]:
user_id_map = {int(user): user_id for user, user_id in user_id_map.items()}
item_id_map = {int(item): item_id for item, item_id in item_id_map.items()}

In [94]:
with open('lightfm_model.pkl', 'wb') as model_file:
    pickle.dump(model_clear, model_file)

# Save the user_id_mapping
with open('model_user_id_map.pkl', 'wb') as user_file:
    pickle.dump(user_id_map, user_file)

# Save the item_id_mapping
with open('model_item_id_map.pkl', 'wb') as item_file:
    pickle.dump(item_id_map, item_file)

    # Save the user_to_name_map
with open('user_to_name_map.pkl', 'wb') as user_name_file:
    pickle.dump(user_to_name_map, user_name_file)

    # Save the item_to_name_map
with open('item_to_name_map.pkl', 'wb') as item_name_file:
    pickle.dump(item_to_name_map, item_name_file)

    # Save the known user interactions
with open('user_item_interactions.pkl', 'wb') as interactions_name_file:
    pickle.dump(known_interactions, interactions_name_file)

    # Save the top_20_items
with open('top_20_items.pkl', 'wb') as top_20_items_file:
    pickle.dump(top_20_items, top_20_items_file)

print("Model and mappings have been saved successfully!")

Model and mappings have been saved successfully!
